In [3]:
import pandas as pd
import numpy as np
import random 
import os

In [4]:
df = pd.read_excel("あらターゲット熟語.xlsx",header=None)
df.columns = ("number","English","Japanese")
df = df.set_index("number")
wrong = df["Japanese"].unique()
wrong_list = pd.DataFrame(wrong)
df.head()

,English,Japanese
number,,
1,a number of ~,いくつもの~;かなり多くの~
2,a piece of ~,1つ[1個;1本;1枚;...]の~
3,a couple of ~,"12つの~;2人の~ 22,3の~"
4,a great[good] deal (of ~),かなりたくさん(の~)
5,plenty of ~,たくさんの~


In [5]:
df.describe()

,English,Japanese
count,1000,1000
unique,951,952
top,be tolerant of ~,~のために
freq,2,5


In [6]:
def testmake(n,start,stop) -> pd.DataFrame:
    #テスト範囲の取得
    test_range = df.iloc[start-1:stop]
    test_sampled = test_range.sample(n = n,replace = False)
    
    #テストの選択肢の作成
    for i in range(4):
        test_sampled["test_{}".format(i+1)] = None
    test_sampled["answer"] = None
    test = test_sampled.iloc[0:0]
  
    #選択肢に正解と問題を入れる    
    for i in range(0,len(test_sampled)):
        test_question = test_sampled.iloc[[i],:].copy()
        answer = random.randint(1,4)
        test_question["test_{}".format(answer)] = test_question["Japanese"]

        test_wrong = wrong_list.sample(n = 4)
        while pd.concat([test_wrong,test_question["Japanese"]]).nunique().sum() <= 4:
            test_wrong = wrong_list.sample(n = 4,replace=False)
        
        for wro in range(1,5):
            if test_question["test_{}".format(wro)].iloc[[0]].isnull().bool() == True:
                test_question["test_{}".format(wro)].iloc[[0]] = test_wrong.iloc[wro-1]
                
        test_question["answer"] = answer
        
        test = pd.concat([test,test_question])
        
    del test["Japanese"]
    return test

In [7]:
test_test = testmake(n = 15,start = 1,stop = 100)
print(len(test_test))
test_test

15


,English,test_1,test_2,test_3,test_4,answer
number,,,,,,
41,work on (~),~にもかかわらず,1(火・明かりなど)を消す 2~を出す,(2つのうちで)一方は~、もう一方は...,1働き続ける 2~を製作する;~に取り組む;~に影響を与える,4
58,differ from ~,~と異なる,~について気難しい[やかましい],A(人)をB(事柄)のことで称賛する[認める],~を抑える;~を食い止める,1
11,so far,例えば,後で,(人が人・考えなど)に同意する;(気候・食物などが)~に合う;~に一致[適合]する,今までのところ,4
98,on time,1待機する;傍観する 2~を支持[擁護]する;(約束など)を守る,~に向いて[生まれついて]いる,それと反対の[に],時間通りに,4
39,get to ~,~を求める,出発する;~を引き起こす,今にも~しようとしている,1(get to +(代)名詞で)~に到着する 2(get to do で)~するようになる...,4
7,close to ~,AをBに課する[押しつける],(仕事・許可など)を申し込む;~を志願[申請]する,~のすぐ近くに;ほぼ~,1(場所・時間)を取る;~を取り上げる 2(仕事・趣味など)を始める,3
29,think of ~,~を使い果たす,~と一緒にいる;~に付き合う,(接続詞的に)...するたびに,1~のことを考える;~しようかなと思う 2~を思いつく 3~を思い出す,4
24,be made of ~,~で成り立っている,~を意識している;~に気づいている,AにBを思い出させる,言うまでもなく,1
80,for example[instance],~に反対している,(病気)にかかる,言い換えれば;つまり,例えば,4


In [80]:
test_test = testmake(1,100,15)
test_test["is_correct"] = None
for i in range(len(test_test)):
    ran = np.random.binomial(n = 1,p = 0.8)
    test_test["is_correct"].iloc[i] = ran

In [81]:
test_test

,English,test_1,test_2,test_3,test_4,answer,is_correct
number,,,,,,,
63,give up ~,1(程度)~するほど...である 2(結果)非常に...なので~する,1(glad;happy;pleased などの前で)とても;大いに 2遺憾[残念]ながら...だ,整頓されて;順調で,~をあきらめる;~を捨てる[やめる],4,1
53,complain about[of] ~,~を除いて;~以外は;~がない[なかった]ならば,途方に暮れて,途方に暮れて;困って;損をして,~について不平を言う;(苦痛など)を訴える,4,1
64,set up ~,~を立[建]てる;~を創設する;~を始める,(結果を表して)~だ、その結果...だ,(~と)交換に,~に夢中である;~が大好きである,1,1
55,call for ~,~を信用する;~の良さ[存在]を信じる,よくても;せいぜい,(署名して)(~に)加わる;(受講などの)届け出をする,1~を必要とする;~を求める 2(人)を誘い[迎え]に行く;(物)を取りに行く,4,1
6,dozens of ~,~に関して(は),~を見つける[見かける],何ダースもの~;数十もの~,~を見て,3,1
46,turn out (~),~と一緒に;~に加えて,(規則・命令など)に従う;~に応じる,1~であることがわかる;(結果的に)~になる 2~を産出する 3(催しなどに)繰り出す,のんびりと構える,3,0
21,be about to do,1~まで 2(It is up to ~で)~次第である;~の責任である 3(be up t...,1(他人から離れて)ひとりぼっちで 2(他人の力を借りずに)独力で,今にも~しようとしている,死ぬまで~;ひどく~,3,1
60,result from ~,~から起こる,~をつかむ;~を捕まえる,1(主語とone'sが一致)いつもの席に着く;特定の地位を占める 2(主語とone'sが不一...,ダイエット中の[で],1,1
42,care for ~,ときどき;ときおり,~どころではない;~からほど遠い,~と調和[一致]して,1~の世話をする 2(否定文・疑問文・条件文で)~を好む;~を望む,4,1


In [82]:
correct_list = pd.DataFrame(test_test.loc[test_test["is_correct"] == 1].index)
miss_list = pd.DataFrame(test_test.loc[test_test["is_correct"] == 0].index)

In [83]:
correct_list

,number
0,63
1,53
2,64
3,55
4,6
5,21
6,60
7,42
8,83
9,69
